In [1]:
#Bibliotecas
import pandas as pd
import numpy as np
import psycopg2 

In [2]:
# Configurações de conexão
config = {
    'host': '10.100.1.86',        # ou o endereço do seu servidor
    'database': 'bd_bonus',
    'user': 'giovanni_aud',
    'password': 'Bonus@2025',
    'port': 5432               # porta padrão do PostgreSQL
}
# Conectar ao banco de dados
conn = psycopg2.connect(**config)
cursor = conn.cursor()


print("Conexão estabelecida com sucesso!")

Conexão estabelecida com sucesso!


In [3]:
sheets = pd.read_csv('sheets.csv')
sheets.columns

Index(['idnfsexterno', 'data_emissao_proposta', 'data_ajustada', 'valor_bonus',
       'rebate', 'reembolso_ipva', 'comissao_vd', 'trade', 'portal', 'varejo',
       'equalizacao', 'trade_in', 'taxa_0', 'ipva', 'wallbox', 'portatil',
       'seguro', 'refaturado', 'em_valid', 'opt_comecial', 'campanha',
       'detalhes'],
      dtype='object')

In [4]:
cursor.execute("SELECT schema_name FROM information_schema.schemata;")
schemas = cursor.fetchall()
for schema in schemas:
    print(schema[0])

byd
dep_union
information_schema
pg_catalog
pg_toast


In [5]:
cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'dep_union';")
tables = cursor.fetchall()
for table in tables:
    print(table[0])

union_view_bq


In [6]:
cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'byd';")
tables_byd = cursor.fetchall()
for table in tables_byd:
    print(table[0])
    

bonus_view
byd_cadastro
db_vendas_byd


In [11]:
cursor.execute("DELETE FROM byd.db_vendas_byd;")
conn.commit()
print("Todas as linhas da tabela db_vendas_byd foram apagadas com sucesso!")

Todas as linhas da tabela db_vendas_byd foram apagadas com sucesso!


##### Desativar FK 

In [10]:
# Desativar todas as foreign keys do banco de dados
# Busca todas as constraints do tipo FOREIGN KEY
cursor.execute("""
SELECT tc.table_schema, tc.table_name, tc.constraint_name 
FROM information_schema.table_constraints AS tc 
WHERE tc.constraint_type = 'FOREIGN KEY'
""")
fks = cursor.fetchall()

# Desativa cada constraint encontrada
for schema, table, constraint in fks:
    sql = f'ALTER TABLE "{schema}"."{table}" DROP CONSTRAINT "{constraint}";'
    try:
        cursor.execute(sql)
        print(f'Constraint {constraint} removida da tabela {schema}.{table}')
    except Exception as e:
        print(f'Erro ao remover constraint {constraint} da tabela {schema}.{table}: {e}')
        conn.rollback()  # Limpa o erro da transação para continuar

conn.commit()
print('Todas as foreign keys foram desativadas (removidas) do banco de dados.')

Constraint fk_idnfsexterno removida da tabela byd.byd_cadastro
Todas as foreign keys foram desativadas (removidas) do banco de dados.


In [9]:
conn.rollback()

In [12]:
# Reativar a foreign key que foi removida
cursor.execute("""
ALTER TABLE "byd"."byd_cadastro" 
ADD CONSTRAINT "fk_idnfsexterno" 
FOREIGN KEY (idnfsexterno) 
REFERENCES byd.db_vendas_byd(idnfsexterno);
""")
conn.commit()
print('Foreign key fk_idnfsexterno foi reativada na tabela byd.byd_cadastro')

Foreign key fk_idnfsexterno foi reativada na tabela byd.byd_cadastro


##### Migrar Data Cadastro_Bonus

In [13]:
cursor.execute("SELECT column_name, data_type FROM information_schema.columns WHERE table_schema = 'byd' AND table_name = 'byd_cadastro';")
columns = cursor.fetchall()
for column in columns:
    print(f"{column[0]}: {column[1]}")

idnfsexterno: text
data_emissao_proposta: date
data_ajustada: date
valor_bonus: numeric
rebate: text
reembolso_ipva: text
comissao_vd: text
trade: text
portal: text
varejo: text
equalizacao: boolean
trade_in: boolean
taxa_0: boolean
ipva: boolean
wallbox: boolean
portatil: boolean
seguro: boolean
refaturado: boolean
em_valid: text
opt_comecial: text
campanha: text
detalhes: text
tipo_transacao: character varying
status_nf: character varying
bonus_utilizado: text


In [20]:
cursor.execute("""
ALTER TABLE "byd"."byd_cadastro" 
ALTER COLUMN data_ajustada TYPE timestamp;
""")
conn.commit()
print('Coluna data_ajustada alterada para tipo timestamp com sucesso!')

Coluna data_ajustada alterada para tipo timestamp com sucesso!


In [19]:
cursor.execute("DROP VIEW byd.bonus_view;")
conn.commit()
conn.rollback()
print('View byd.bonus_view foi removida com sucesso!')

View byd.bonus_view foi removida com sucesso!


In [23]:
conn.rollback()

In [ ]:
# 1. Descobrir views que usam a coluna dta_processamento
cursor.execute("""
SELECT table_schema, table_name, view_definition 
FROM information_schema.views 
WHERE view_definition ILIKE '%dta_processamento%'
  AND table_schema = 'byd';
""")
views = cursor.fetchall()
for schema, view, definition in views:
    print(f"View encontrada: {schema}.{view}")

# 2. Remover as views encontradas (exemplo para uma view chamada bonus_view)
try:
    cursor.execute("DROP VIEW IF EXISTS byd.bonus_view;")
    conn.commit()
    print('View byd.bonus_view removida!')
except Exception as e:
    print(f'Erro ao remover a view: {e}')
    conn.rollback()

# 3. Alterar o tipo da coluna
try:
    cursor.execute("""
    ALTER TABLE "byd"."db_vendas_byd" 
    ALTER COLUMN dta_processamento TYPE timestamp;
    """)
    conn.commit()
    print('Coluna dta_processamento alterada para tipo timestamp com sucesso!')
except Exception as e:
    print(f'Erro ao alterar a coluna: {e}')
    conn.rollback()

# 4. (Opcional) Recriar a view removida aqui, se necessário

DatatypeMismatch: column "dta_processamento" cannot be cast automatically to type timestamp without time zone
HINT:  You might need to specify "USING dta_processamento::timestamp without time zone".


In [24]:
cursor.execute("SELECT column_name, data_type FROM information_schema.columns WHERE table_schema = 'byd' AND table_name = 'db_vendas_byd';")
columns = cursor.fetchall()
for column in columns:
    print(f"{column[0]}: {column[1]}")

sistema: text
idnfsexterno: text
data_atual: text
data_atual_inv: text
val_modalidade: text
sinal: text
revenda: text
empresa: text
lucbru: text
chv_empresa: text
bandeira_bi: text
ordem_bandeira: text
nome_empresa: text
numero_nota_fiscal: text
serie_nota_fiscal: text
dta_entrada_saida: text
status_nf: text
dta_processamento: text
dta_cancelamento: text
tipo_transacao: text
des_tipo_transacao: text
tipo: text
subtipo_transacao: text
contador: text
cod_cliente: text
nome_cliente: text
cpf: text
cnpj: text
fisico_juridico: text
bairro_entrega: text
municipio_entrega: text
uf_entrega: text
cod_mun: text
vendedor: text
nome_vendedor: text
proposta: text
situacao: text
veiculo: text
nome_departamento_veiculo: text
novo_usado: text
chassi: text
placa: text
ano_fabricacao: text
ano_modelo: text
modelo: text
des_modelo: text
marca: text
codfamilia: text
familia: text
bonus: text
val_icms: text
val_ipi: text
imposto_importacao: text
val_pis: text
val_cofins: text
nota_origem_devolucao: text
dt